In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt

In [4]:
symbol = input("Enter stock symbol (e.g., RELIANCE, TCS, INFY): ").upper()
exchange = input("Enter exchange (NS for NSE, BO for BSE): ").upper()
ticker = f"{symbol}.{exchange}"

start_date = input("Enter start date (YYYY-MM-DD): ")
end_date = datetime.today().strftime('%Y-%m-%d')
data = yf.download(ticker, start=start_date, end=end_date)

Enter stock symbol (e.g., RELIANCE, TCS, INFY):  RPOWER
Enter exchange (NS for NSE, BO for BSE):  NS
Enter start date (YYYY-MM-DD):  2025-01-01


C:\Users\abali\AppData\Local\Temp\ipykernel_23612\915661245.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


In [5]:
stock = yf.Ticker(ticker)

# Fetch fundamentals
eps = stock.info.get("trailingEps", None)            # Earnings per Share
pe_ratio = stock.info.get("trailingPE", None)        # Price to Earnings ratio
roe = stock.info.get("returnOnEquity", None)         # Return on Equity (fraction, e.g., 0.18 = 18%)
de_ratio = stock.info.get("debtToEquity", None)      # Debt-to-Equity ratio

print("\nFundamental Metrics:")
print(f"EPS: {eps}")
print(f"P/E Ratio: {pe_ratio}")
print(f"ROE: {roe}")
print(f"Debt-to-Equity: {de_ratio}")


Fundamental Metrics:
EPS: 7.55
P/E Ratio: 5.94702
ROE: None
Debt-to-Equity: 92.752


In [6]:
# --- Moving Averages ---
data["MA50"] = data["Close"].rolling(50).mean()
data["MA200"] = data["Close"].rolling(200).mean()

# --- RSI Calculation ---
delta = data["Close"].diff()
gain = (delta.where(delta > 0, 0)).rolling(14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(14).mean()

rs = gain / loss
data["RSI"] = 100 - (100 / (1 + rs))

print("\nLatest Technical Indicators:")
print(f"50-day MA: {data['MA50'].iloc[-1]}")
print(f"200-day MA: {data['MA200'].iloc[-1]}")
print(f"RSI: {data['RSI'].iloc[-1]}")


Latest Technical Indicators:
50-day MA: 57.47499977111816
200-day MA: nan
RSI: 48.95508833849141


In [23]:
# 52-week high/low are available in .info
ticker_2 = yf.Ticker("AAPL")

high_52w = ticker_2.info.get("fiftyTwoWeekHigh")
low_52w = ticker_2.info.get("fiftyTwoWeekLow")
current_price = data["Close"].iloc[-1].item()

print("52-week high/low:")
print("52-Week High:", high_52w)
print("52-Week Low:", low_52w)
print("current_price", current_price)

52-week high/low:
52-Week High: 260.1
52-Week Low: 169.21
current_price 44.72999954223633


In [12]:
# Revenue growth (yearly) is available in .info
rev_growth = ticker_2.info.get("revenueGrowth")   # usually as decimal (0.12 = 12%)
if rev_growth:
    rev_growth = rev_growth * 100   # convert to %
    
print("Revenue Growth %:", rev_growth)

Revenue Growth %: 9.6


In [13]:
# Quarterly financials (rows = line items, cols = periods)
fin = ticker_2.quarterly_financials
print(fin)

# Example: Net Income growth (QoQ)
net_income = fin.loc["Net Income"]

if len(net_income) >= 2:
    profit_growth = ((net_income[0] - net_income[1]) / net_income[1]) * 100
    print("Profit Growth % (QoQ):", profit_growth)

                                                      2025-06-30  \
Tax Effect Of Unusual Items                         0.000000e+00   
Tax Rate For Calcs                                  1.639970e-01   
Normalized EBITDA                                   3.103200e+10   
Net Income From Continuing Operation Net Minori...  2.343400e+10   
Reconciled Depreciation                             2.830000e+09   
Reconciled Cost Of Revenue                          5.031800e+10   
EBITDA                                              3.103200e+10   
EBIT                                                2.820200e+10   
Normalized Income                                   2.343400e+10   
Net Income From Continuing And Discontinued Ope...  2.343400e+10   
Total Expenses                                      6.583400e+10   
Total Operating Income As Reported                  2.820200e+10   
Diluted Average Shares                              1.494818e+10   
Basic Average Shares                            

C:\Users\abali\AppData\Local\Temp\ipykernel_23612\495297547.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  profit_growth = ((net_income[0] - net_income[1]) / net_income[1]) * 100


In [14]:
fin_y = ticker_2.financials
net_income_y = fin_y.loc["Net Income"]

if len(net_income_y) >= 2:
    profit_growth_y = ((net_income_y[0] - net_income_y[1]) / net_income_y[1]) * 100
    print("Profit Growth % (YoY):", profit_growth_y)

Profit Growth % (YoY): -3.3599670086086912


C:\Users\abali\AppData\Local\Temp\ipykernel_23612\168983890.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  profit_growth_y = ((net_income_y[0] - net_income_y[1]) / net_income_y[1]) * 100


In [25]:
def evaluate_stock(eps, pe_ratio, roe, de_ratio, ma50, ma200, rsi, high_52w=None, low_52w=None, current_price=None, revenue_growth=None, profit_growth=None):
    score = 0

    # Fundamentals
    if eps and eps > 0: score += 1
    if pe_ratio and pe_ratio < 20: score += 1
    if roe and roe > 0.15: score += 1
    if de_ratio and de_ratio < 1: score += 1

    # Technicals
    if ma50 > ma200: score += 1
    if rsi and rsi < 70 and rsi > 30: score += 1

     # Price near 52-week high/low (good if closer to high than low)
    if high_52w and low_52w and current_price:
        if current_price > (low_52w + (0.75 * (high_52w - low_52w))):  
            score += 1  

    # Revenue Growth positive
    if rev_growth and rev_growth > 0:  
        score += 1  

    # Profit Growth positive
    if profit_growth and profit_growth > 0:  
        score += 1  

    # Final decision
    if score >= 7:
        return " Strong Buy"
    elif score >= 5:
        return " Buy / Hold"
    else:
        return " Avoid / Weak"

In [26]:
result = evaluate_stock(eps, pe_ratio, roe, de_ratio, data["MA50"].iloc[-1], data["MA200"].iloc[-1], data["RSI"].iloc[-1], high_52w, low_52w, current_price,rev_growth, profit_growth)
print("Final Recommendation:", result)

Final Recommendation:  Avoid / Weak
